In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

In [2]:
# Load the dataframe.
df_train = pd.read_csv('../data/train.csv')

In [3]:
# Group by 'user_session' and create new features.
agg_df = df_train.groupby('user_session').agg(
    unique_event_types=('event_type', 'nunique'),
    unique_products=('product_id', 'nunique'),
    unique_categories=('category_id', 'nunique'),
    event_count=('event_time', 'count'),
    user_id=('user_id', 'first'),
    session_value=('session_value', 'first')
).reset_index()

In [4]:
# Prepare features (X) and target (y)
features = ['unique_event_types', 'unique_products', 'unique_categories', 'event_count']
X = agg_df[features]
y = agg_df['session_value']

In [5]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
# Initialize and train the Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

In [7]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate MSE and RMSE
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"Modelin Ortalama Kare Hatası (MSE): {mse:.4f}")
print(f"Modelin Kök Ortalama Kare Hatası (RMSE): {rmse:.4f}")

Modelin Ortalama Kare Hatası (MSE): 1636.4275
Modelin Kök Ortalama Kare Hatası (RMSE): 40.4528


In [8]:
# Adım 4: test.csv dosyasını yükle ve aynı özellik mühendisliğini yap
print("\ntest.csv dosyası yükleniyor ve tahminler için hazırlanıyor...")
try:
    df_test = pd.read_csv('../data/test.csv')
except FileNotFoundError:
    print("Hata: test.csv dosyası bulunamadı. Lütfen dosyayı yükleyip tekrar deneyin.")
    exit()

agg_df_test = df_test.groupby('user_session').agg(
    unique_event_types=('event_type', 'nunique'),
    unique_products=('product_id', 'nunique'),
    unique_categories=('category_id', 'nunique'),
    event_count=('event_time', 'count')
).reset_index()

# Adım 5: Eğitilmiş modeli kullanarak tahminleri yap
print("Tahminler yapılıyor...")
X_test = agg_df_test[features]
predictions = model.predict(X_test)

# Adım 6: Tahminleri orijinal test veri çerçevesine ekle
agg_df_test['predicted_session_value'] = predictions

# Adım 7: Tahminleri yeni bir CSV dosyasına kaydet
print("Tahminler 'test_predictions.csv' dosyasına kaydediliyor...")
agg_df_test.to_csv('test_predictions.csv', index=False)
print("İşlem tamamlandı. Tahminler, 'test_predictions.csv' dosyasında yer almaktadır.")

# Adım 8: Tahminlerin bir önizlemesini göster
print("\nTahminlerin ilk 5 satırı:")
print(agg_df_test.head())


test.csv dosyası yükleniyor ve tahminler için hazırlanıyor...
Tahminler yapılıyor...
Tahminler 'test_predictions.csv' dosyasına kaydediliyor...
İşlem tamamlandı. Tahminler, 'test_predictions.csv' dosyasında yer almaktadır.

Tahminlerin ilk 5 satırı:
     user_session  unique_event_types  unique_products  unique_categories  \
0  SESSION_000000                   2               27                 19   
1  SESSION_000013                   1                2                  2   
2  SESSION_000022                   1                1                  1   
3  SESSION_000024                   1                1                  1   
4  SESSION_000025                   2                3                  2   

   event_count  predicted_session_value  
0           28               327.753694  
1            2                44.131416  
2            1                30.699615  
3            1                30.699615  
4            3                57.610650  


In [9]:
import pandas as pd

# Load the previously created predictions file.
try:
    df_predictions = pd.read_csv('test_predictions.csv')
except FileNotFoundError:
    print("Hata: test_predictions.csv dosyası bulunamadı. Lütfen önce tahminleri oluşturun.")
    exit()

# Select only the user_session and predicted_session_value columns.
df_final_predictions = df_predictions[['user_session', 'predicted_session_value']]

# Save the new DataFrame to a new CSV file.
output_filename = 'final_test_predictions.csv'
df_final_predictions.to_csv(output_filename, index=False)

print(f"Başarıyla `user_session` ve `predicted_session_value` sütunlarını {output_filename} dosyasına kaydettim.")

# Display the head of the new dataframe.
print(df_final_predictions.head())

Başarıyla `user_session` ve `predicted_session_value` sütunlarını final_test_predictions.csv dosyasına kaydettim.
     user_session  predicted_session_value
0  SESSION_000000               327.753694
1  SESSION_000013                44.131416
2  SESSION_000022                30.699615
3  SESSION_000024                30.699615
4  SESSION_000025                57.610650
